## ***Zero Shot LLM Evaluation.ipynb***

In [1]:
#imporitng libraries
import pandas as pd
from transformers import pipeline
import torch
from tqdm import tqdm

In [6]:
#loading open source model (t5 xl)
model  = pipeline('text2text-generation',model='google/flan-t5-xl',device=0)


config.json: 0.00B [00:00, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.45G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


In [4]:
#test data loading
test_path = "/content/drive/MyDrive/MScDissertation-Sonu/data/processed/test_processed.csv"
df_test = pd.read_csv(test_path)


In [7]:
#crafting the prompt template
def create_prompt(dialogue,section_header):
  prompt = f"""
      Please generate a concise summary for the following section of a clinical note based on the doctor-patient dialogue.

    Section Header: {section_header}

    Dialogue:
    {dialogue}

    Summary:
    """
  return prompt.strip()


In [8]:
#generating_summaries
summaries = []
output_path = "/content/drive/MyDrive/MScDissertation-Sonu/results/flan_t5_xl_generated_summaries.csv"
for i,row in tqdm(df_test.iterrows(),total=df_test.shape[0]):
  full_input = row['input_text']
  header_part, dialogue_part = full_input.split(" | dialogue:-")
  section_header = header_part.replace('summarize for section :- ','')
  dialogue = dialogue_part
  prompt = create_prompt(dialogue,section_header)

  response = model(prompt,max_length=256,min_length=8,
                   no_repeat_ngram_size=3,encoder_no_repeat_ngram_size=3,
                   repetition_penalty=3.5,num_beams=4,early_stopping=True,)
  summary = response[0]['generated_text']
  summaries.append(summary)


  5%|▌         | 10/200 [00:10<02:38,  1.20it/s]You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Both `max_new_tokens` (=256) and `max_length`(=256) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
 16%|█▌        | 31/200 [00:29<02:09,  1.30it/s]Token indices sequence length is longer than the specified maximum sequence length for this model (835 > 512). Running this sequence through the model will result in indexing errors
Both `max_new_tokens` (=256) and `max_length`(=256) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
100%|██████████| 200/200 [03:26<00:00,  1.03s/it]


In [9]:
df_test['llm_generated_summary'] = summaries
df_test.to_csv(test_path,index=False)